## Forecasting Water Demand

In [ ]:
import pickle
import warnings
from math import sqrt

import matplotlib as mpl
import numpy as np
import pandas as pd  # Basic library for all of our dataset operations
import pmdarima as pm
import tensorflow as tf
import xgboost as xgb
from xgboost import XGBRegressor as xgbr

from matplotlib import pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV, cross_val_score, KFold, GridSearchCV
from sklearn.metrics import make_scorer, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm

# We will use deprecated models of statmodels which throw a lot of warnings to use more modern ones
warnings.filterwarnings("ignore")


# Extra settings
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)
plt.style.use('bmh')
mpl.rcParams['axes.labelsize'] = 14
mpl.rcParams['xtick.labelsize'] = 12
mpl.rcParams['ytick.labelsize'] = 12
mpl.rcParams['text.color'] = 'k'
mpl.rcParams['figure.figsize'] = 18, 8

print(tf.__version__)

### 1. Prepare data before Forecasting

In [ ]:
# set city name
city_data = "1_goyang_city.xlsx"
city_name = "GoYang-City"

In [ ]:
# Read excel file using pandas
df = pd.read_excel(open(f"../1_feature_selection_all/data/{city_data}", 'rb'), sheet_name="training", header=4, index_col=0)
# Remove unnecessary columns for this analysis
water_demand = df.iloc[2:]
# Change Date Format and Set Date as index
water_demand.index = pd.to_datetime(water_demand.index.str.strip(), format='%Y-%m')
water_demand.index.name = "date"
# Change data format from "Object" to "Float"
water_demand["water_supply"] = water_demand.water_supply.astype(float)
water_demand["Total_Population"] = water_demand.Total_Population.astype(float)
# Delete unnecessary columns 
water_demand.drop(columns=water_demand.columns[19:21], inplace=True)
water_demand.drop(columns=water_demand.columns[22:23], inplace=True)
# Select clean data
water_demand = water_demand.loc["2010-01-01":]
water_demand

In [ ]:
feature_selection = pd.read_csv("GoYang-City_feature_selection_ranking.csv")
feature_selection

In [ ]:
list(feature_selection[feature_selection.columns[1]][0:5].values)

In [ ]:
water_demand_re = water_demand[list(feature_selection[feature_selection.columns[1]][0:5].values)]
water_demand_re["water_supply"] = water_demand["water_supply"].values
water_demand = water_demand_re
water_demand

In [ ]:
# We split our dataset to be able to evaluate our models

resultsDict = {}
predictionsDict = {}

#air_pollution = pd.read_csv('datasets/air_pollution.csv', parse_dates=['date'])
#air_pollution.set_index('date', inplace=True)

split_date = '2018-12-01'
df_training = water_demand.loc[water_demand.index <= split_date]
df_test = water_demand.loc[water_demand.index > split_date]
print(f"{len(df_training)} days of training data \n {len(df_test)} days of testing data ")

df_training.to_csv(f'training_{city_name}.csv')
df_test.to_csv(f'test_{city_name}.csv')

In [ ]:
# ADD time features to our model
def create_time_features(df, target=None):
    """
    Creates time series features from datetime index
    """
    df['date'] = df.index
    df['hour'] = df['date'].dt.hour
    df['dayofweek'] = df['date'].dt.dayofweek
    df['quarter'] = df['date'].dt.quarter
    df['month'] = df['date'].dt.month
    df['year'] = df['date'].dt.year
    df['dayofyear'] = df['date'].dt.dayofyear
    df['sin_day'] = np.sin(df['dayofyear'])
    df['cos_day'] = np.cos(df['dayofyear'])
    df['dayofmonth'] = df['date'].dt.day
    #df['weekofyear'] = df['date'].dt.weekofyear
    X = df.drop(['date'], axis=1)
    if target:
        y = df[target]
        X = X.drop([target], axis=1)
        return X, y

    return X

In [ ]:
X_train_df, y_train = create_time_features(
    df_training, target='water_supply')
X_test_df, y_test = create_time_features(df_test, target='water_supply')
scaler = StandardScaler()
scaler.fit(X_train_df)  # No cheating, never scale on the training+test!
X_train = scaler.transform(X_train_df)
X_test = scaler.transform(X_test_df)

X_train_df = pd.DataFrame(X_train, columns=X_train_df.columns)
X_test_df = pd.DataFrame(X_test, columns=X_test_df.columns)

### 2. Forecasting Methods

#### 2.1 Randomforest

#### 2.1.1 적정 n_estimator 검토

In [ ]:
# R2와 RMSE 점수와 트리개수를 담을 리스트
r2_scores = []
rmse_scores = []
estimators = []

# warm_start=True로 RandomForestRegressor 객체를 만듦
rf = RandomForestRegressor(warm_start=True, n_jobs=-1, random_state=2)

# 시작트리개수
est = 10

In [ ]:
# 0~31까지 반복
for i in range(41):
    
    # n_estimator를 est로 설정
    rf.set_params(n_estimators=est)
    
    # 인구조사 데이터셋으로 훈련
    rf.fit(X_train, y_train)
    
    # RMSE 값을 계산
    rmse = mean_squared_error(y_test, rf.predict(X_test), squared=False)
    
    # rmse와 est를 리스트에 추가함
    rmse_scores.append(rmse)
    estimators.append(est)
    
    # 트리를 25개씩 늘림
    est += 25

# 그래프 크기
plt.figure(figsize=(15,7))

# estimators와 oob_scores를 그래프로 그림
plt.plot(estimators, rmse_scores)

for i in range(len(estimators)):
    plt.annotate(str(estimators[i]), xy=(estimators[i], rmse_scores[i]), fontsize=8)
    
# 축 레이블을 설정
plt.xlabel("Number of Trees", fontsize=15)
plt.ylabel("RMSE", fontsize=15)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)

# 제목을 출력
plt.title(f"Optimal n_estimator evaluation to apply Random Forest ({city_name})", fontsize=20)
plt.grid(True)
plt.savefig(f"./{city_name}_optimal_n_estimator_RF_fs.png", bbox_inches='tight')
plt.show()

#### 2.1.2 교차검증을 통한 모델검증

In [ ]:
n_estimator = 250
rf = RandomForestRegressor(n_estimators=n_estimator, warm_start=True, n_jobs=-1, random_state=2)
scores = cross_val_score(rf, X_train, y_train, scoring="neg_mean_squared_error", cv=10)
rmse = np.sqrt(-scores)
print('RMSE:', np.round(rmse,3))
print('RMSE 평균: %0.3f' % (rmse.mean()))

#### 2.1.3 하이퍼파라미터 튜닝

 * 하이퍼파라미터 튜닝을 위한 함수 정의

In [ ]:
def randomized_search_reg(params, runs=16, reg=RandomForestRegressor(n_estimators=n_estimator, warm_start=True, random_state=2, n_jobs=-1)):
    rand_reg = RandomizedSearchCV(reg, params, n_iter=runs, scoring='neg_mean_squared_error', cv=10,
                                  n_jobs=-1, random_state=2)
    rand_reg.fit(X_train, y_train)
    best_model = rand_reg.best_estimator_
    best_params = rand_reg.best_params_
    print("최상의 매개변수:", best_params)
    best_score = np.sqrt(-rand_reg.best_score_)
    print("훈련점수: {:.3f}".format(best_score))
    y_pred = best_model.predict(X_test)
    from sklearn.metrics import mean_squared_error as MSE
    rmse_test = mean_squared_error(y_test, y_pred, squared=False)
    print("테스트 세트 점수: {:.3f}".format(rmse_test))
    return best_params

 * 적정 하이퍼파라미터 튜닝을 위한 하이퍼파라미터 값 설정

In [ ]:
best_params = randomized_search_reg(
              params={'max_depth':[None, 2, 4, 6, 8, 10, 20],
                      'max_leaf_nodes':[10, 15, 20, 25, 30, 35, 40, 45, 50, None],
                      'max_features':['auto', 0.8, 0.7, 0.6, 0.5, 0.4],
                      'min_impurity_decrease':[0.0, 0.01, 0.05, 0.10, 0.15, 0.2]})

#### 2.1.4 최적 모델적용 및 Test 데이터에 대한 성능평가

In [ ]:
# warm_start=True로 RandomForestRegressor 객체를 만듦
rf = RandomForestRegressor(n_estimators=n_estimator, random_state=2, n_jobs=-1, 
                           max_depth=best_params['max_depth'], max_leaf_nodes=best_params['max_leaf_nodes'],
                           max_features=best_params['max_features'], 
                           min_impurity_decrease=best_params['min_impurity_decrease'])

In [ ]:
rf.fit(X_train, y_train)

# R2과 RMSE 값을 계산
train_rf_r2 = r2_score(y_train, rf.predict(X_train))
train_rf_rmse = mean_squared_error(y_train, rf.predict(X_train), squared=False)
test_rf_r2 = r2_score(y_test, rf.predict(X_test))
test_rf_rmse = mean_squared_error(y_test, rf.predict(X_test), squared=False)

print("RF Train R2: {:.2f}".format(train_rf_r2))
print("RF Train RMSE: {:.2f}".format(train_rf_rmse))
print("RF Test R2: {:.2f}".format(test_rf_r2))
print("RF Test RMSE: {:.2f}".format(test_rf_rmse))

#### 2.1.5 Plotting

In [ ]:
fig = plt.figure(figsize=(20, 6))
ax = fig.add_subplot(111)

ax.scatter(water_demand["water_supply"].index, water_demand["water_supply"], label='Water Supply Observation', c='black', s=70)  # Plot some data on the axes.
ax.plot(water_demand["water_supply"].index, np.append(rf.predict(X_train), rf.predict(X_test)), linewidth=3, label='Water Demand Forecasting (Random Forest)')  # Plot more data on the axes...

#ax.plot(water_demand["water_supply"].index, water_demand["water_supply"], linewidth=3, label='Water Supply Observation')  # Plot some data on the axes.
#ax.scatter(water_demand["water_supply"].index, np.append(rf.predict(X_train), rf.predict(X_test)), c='r', s=70, label='Water Demand Forecasting (Random Forest)')  # Plot more data on the axes...
ax.plot([water_demand["water_supply"].index[108], water_demand["water_supply"].index[108]], [water_demand["water_supply"].min(), water_demand["water_supply"].max()], linewidth=5, linestyle="dashed")
#ax.text(water_demand["water_supply"].index[90], water_demand["water_supply"].max()-10000, "Train", horizontalalignment='left', size=20)
#ax.text(water_demand["water_supply"].index[125], water_demand["water_supply"].max()-10000, "Test", horizontalalignment='right', size=20)
ax.text(water_demand["water_supply"].index[100], water_demand["water_supply"].max()-100000, f"Train Performance\n$R^2$:{train_rf_r2:.2f}, RMSE:{train_rf_rmse:.0f}", horizontalalignment='right', size=20)
ax.text(water_demand["water_supply"].index[115], water_demand["water_supply"].max()-100000, f"Test Performance\n$R^2$:{test_rf_r2:.2f}, RMSE:{test_rf_rmse:.0f}", horizontalalignment='left', size=20)

ax.set_xlabel('Date', fontsize=20)  # Add an x-label to the axes.
ax.set_ylabel('Water Supply', fontsize=20)  # Add a y-label to the axes.
#ax.set_title(f"Performance Evaluation using Random Forest with All variables ({city_name})", fontsize=20)  # Add a title to the axes.
ax.legend(fontsize=20)  # Add a legend.
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
ax.grid()
ax.grid()
plt.savefig(f"./f{city_name}_performance_plot_rf_fs_variable.png", bbox_inches='tight')

In [ ]:
water_demand["random_forest"] = np.append(rf.predict(X_train), rf.predict(X_test))
rf_result = water_demand[["water_supply", "random_forest"]]
rf_result.to_csv(f"{city_name}_RF_result_selection.csv")

### 2.2 XGBoost

#### 2.2.1 적정 n_estimator 검토

In [ ]:
kfold = KFold(n_splits=5, shuffle=True, random_state=2)

def n_estimators(model):
    eval_set = [(X_test, y_test)]
    eval_metric = "rmse"
    model.fit(X_train, y_train, eval_metric=eval_metric,
              eval_set=eval_set, early_stopping_rounds=100)
    y_pred = model.predict(X_test)
    rmse = mean_squared_error(y_test, y_pred)**0.5
    return rmse
    
n_estimators(xgbr(n_estimators=5000))

#### 2.2.2 하이퍼파라미터 튜닝

In [ ]:
def grid_search(params, reg=xgbr()):
    grid_reg = GridSearchCV(reg, params, scoring='neg_mean_squared_error', cv=kfold)
    grid_reg.fit(X_train, y_train)
    best_params = grid_reg.best_params_
    print("최상의 매개변수:", best_params)
    best_score = np.sqrt(-grid_reg.best_score_)
    print("최상의 점수:", best_score)
    return best_params
best_params = grid_search(params={'max_depth':[1, 2, 3, 4, 5, 6, 7, 8],
                                  'n_estimators':[1, 3, 10, 20, 30, 40]})

#### 2.2.3 최적 모델적용 및 Test 데이터에 대한 성능평가

In [ ]:
model = xgbr(max_depth=best_params['max_depth'],
             n_estimators=best_params['n_estimators'])
model.fit(X_train, y_train)

In [ ]:
# R2과 RMSE 값을 계산
train_xgb_r2 = r2_score(y_train, model.predict(X_train))
train_xgb_rmse = mean_squared_error(y_train, model.predict(X_train), squared=False)
test_xgb_r2 = r2_score(y_test, model.predict(X_test))
test_xgb_rmse = mean_squared_error(y_test, model.predict(X_test), squared=False)

print("XGB Train R2: {:.2f}".format(train_xgb_r2))
print("XGB Train RMSE: {:.2f}".format(train_xgb_rmse))
print("XGB Test R2: {:.2f}".format(test_xgb_r2))
print("XGB Test RMSE: {:.2f}".format(test_xgb_rmse))

#### 2.2.4 Plotting

In [ ]:
fig = plt.figure(figsize=(20, 6))
ax = fig.add_subplot(111)

ax.scatter(water_demand["water_supply"].index, water_demand["water_supply"], label='Water Supply Observation', c='black', s=70)  # Plot some data on the axes.
ax.plot(water_demand["water_supply"].index, np.append(model.predict(X_train), model.predict(X_test)), linewidth=3, label='Water Demand Forecasting (XGBoost)')  # Plot more data on the axes...
#ax.plot(water_demand["water_supply"].index, water_demand["water_supply"], linewidth=3, label='Water Supply Observation')  # Plot some data on the axes.
#ax.scatter(water_demand["water_supply"].index, np.append(model.predict(X_train), model.predict(X_test)), c='r', s=70, label='Water Demand Forecasting (XGBoost)')  # Plot more data on the axes...
ax.plot([water_demand["water_supply"].index[108], water_demand["water_supply"].index[108]], [water_demand["water_supply"].min(), water_demand["water_supply"].max()], linewidth=5, linestyle="dashed")
#ax.text(water_demand["water_supply"].index[90], water_demand["water_supply"].max()-10000, "Train", horizontalalignment='left', size=20)
#ax.text(water_demand["water_supply"].index[125], water_demand["water_supply"].max()-10000, "Test", horizontalalignment='right', size=20)
ax.text(water_demand["water_supply"].index[100], water_demand["water_supply"].max()-100000, f"Train Performance\n$R^2$:{train_xgb_r2:.2f}, RMSE:{train_xgb_rmse:.0f}", horizontalalignment='right', size=20)
ax.text(water_demand["water_supply"].index[115], water_demand["water_supply"].max()-100000, f"Test Performance\n$R^2$:{test_xgb_r2:.2f}, RMSE:{test_xgb_rmse:.0f}", horizontalalignment='left', size=20)

ax.set_xlabel('Date', fontsize=20)  # Add an x-label to the axes.
ax.set_ylabel('Water Supply', fontsize=20)  # Add a y-label to the axes.
#ax.set_title(f"Performance Evaluation using Random Forest with All variables ({city_name})", fontsize=20)  # Add a title to the axes.
ax.legend(fontsize=20)  # Add a legend.
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
ax.grid()
ax.grid()
plt.savefig(f"./{city_name}_performance_plot_xgb_fs_variable.png", bbox_inches='tight')

In [ ]:
water_demand["xgboost"] = np.append(model.predict(X_train), model.predict(X_test))
rf_result = water_demand[["water_supply", "xgboost"]]
rf_result.to_csv(f"{city_name}_XGBoost_result_selection.csv")